In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
path_corpus_data = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
figer_path = base_dir + "FIGER/"

Mounted at /content/drive/


In [2]:
only_corpus=False

In [3]:
if not only_corpus:
  open(path_corpus_data + 'counters_figer.json', 'w').close()
  open(path_corpus_data + 'types_counters_figer.json', 'w').close()

In [4]:
import json

with open(figer_path + "train.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

In [5]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  1505765 elementi


In [6]:
#ci sono 36 frasi che hanno le mentions vuote, le cancello
indici = []
for i,elem in enumerate(examples_train):
  if len(elem['mentions'])==0:
     indici.append(i)

pop=0
for num in indici:
  del examples_train[num-pop]
  pop+=1

In [7]:
#examples_train=examples_train[0:10000]

In [8]:
entire_dataset_list = iter(examples_train)

In [9]:
def get_permutations(i, threshold,generating_only_corpus):
  # calcolo le permutazioni sulla base del dizionario, se sforano il treshold
  # non le esplodo
  d = sorted(i, key=operator.itemgetter("start"))
  d = {k: list(v) for k, v in itertools.groupby(d, key=operator.itemgetter("start"))}

  all_list=[]
  for key, value in d.items():
    all_list.append(value)

  p = list(itertools.product(*all_list))
  sum=0
  #se sto generando solo il corpus e non mi interessa delle occorrenze
  if generating_only_corpus:
    for per in p:
      prod=1
      for i in per:
        prod*= len(i)
        #qui se prod già sfora treshold posso fermarmi tanto non mi serve sapere quante permutazioni genererebbe
        if prod>treshold:
          return [], [], sum
      sum+=prod
      #qui se sum già sfora treshold posso fermarmi tanto non mi serve sapere quante permutazioni genererebbe
      if sum>treshold:
        return [], [], sum
    
  else:
    for per in p:
      prod=1
      for i in per:
        prod*= len(i)
        #qui se prod già sfora treshold posso fermarmi
      sum+=prod
      #qui se sum già sfora treshold posso fermarmi

  if (sum>threshold):
    return [], [], sum

  else:
    total_perm=[]
    count_perm=0
    for per in p:
      mini_list = []
      for lista_labels in per:
        mini_list.append(lista_labels['labels'])

      p2 = list(itertools.product(*mini_list))
      total_perm.append(p2)

    return total_perm.copy(), p.copy(), sum

In [10]:
import operator
import itertools
from tqdm import tqdm
from gensim.parsing.preprocessing import remove_stopwords
import json
treshold=20
perm_a_priori=[]
perm_effettive=[]
sottostime=[]
lunghezze={
    'frasi_considerate':0,
    'frasi_non_considerate':0,
    'tot_permutazioni':0,
}

fp = open(path_corpus_data + 'corpus_data_entire_figer.txt', "w")
restricted_set = {}

for num,i in enumerate(tqdm(entire_dataset_list)):
  if not only_corpus:
    dizionario_permutations = {'pos':num, 'num': 0,'istanza':i}
  istance = i['mentions'][:]
  if len(istance)>1:
    
    permutations_2, permutations, num_permutations = get_permutations(istance, treshold,only_corpus)
    if not only_corpus:
      dizionario_permutations['num'] = num_permutations
    if (not permutations):
      lunghezze['frasi_non_considerate']+=1
    else:
      lunghezze['frasi_considerate']+=1
      lunghezze['tot_permutazioni']+=num_permutations
    iteri=0

    for p_1 in permutations:
      for p_2 in iter(permutations_2[iteri]):
        
        phrase = i['tokens'][:]
        num_pop=0
        for h in range(0, len(p_2)):
          start = permutations[iteri][h]['start']
          end = permutations[iteri][h]['end']

          phrase[start-num_pop:end-num_pop] = [p_2[h].upper()]
          if ((end-start)>1):
            num_pop += (end - start) - 1
          if not only_corpus:
            if (p_2[h].upper() not in restricted_set):
              restricted_set[p_2[h].upper()]=1
            else:
              restricted_set[p_2[h].upper()]+=1

        
        fp.write((remove_stopwords(' '.join(phrase)) + '\n'))

      iteri+=1
    
  else:
    num_permutations=len(istance[0]['labels'])
    if not only_corpus:
      dizionario_permutations['num'] = num_permutations
    lunghezze['tot_permutazioni']+= num_permutations
    lunghezze['frasi_considerate']+=1

    for k in istance[0]['labels']:
      phrase = i['tokens'][:]
      if (i['mentions'][0]['end'] - i['mentions'][0]['start'] ==1):          
        phrase[i['mentions'][0]['start']] = k.upper()
      else:
        phrase[i['mentions'][0]['start']:i['mentions'][0]['end']] = [k.upper()]
      if not only_corpus:
        if (k.upper() not in restricted_set):
          restricted_set[k.upper()]=1
        else:
          restricted_set[k.upper()]+=1

      fp.write((remove_stopwords(' '.join(phrase)) + '\n'))
  if not only_corpus:
    with open(path_corpus_data + 'counters_figer.json', 'a') as fp_1:
        json.dump(dizionario_permutations, fp_1)

fp.close()

1505729it [00:50, 29593.73it/s]


In [11]:
 if not only_corpus:
  with open(path_corpus_data + 'types_counters_figer.json', 'a') as fp_3:
        json.dump(restricted_set, fp_3)

In [13]:
print(f"frasi considerate con treshold {treshold}: {lunghezze['frasi_considerate']}")
print(f"frasi non considerate con treshold {treshold}: {lunghezze['frasi_non_considerate']}")
print(f"media di permutazioni per frase considerata {lunghezze['tot_permutazioni']/lunghezze['frasi_considerate']}")

frasi considerate con treshold 20: 1219463
frasi non considerate con treshold 20: 286266
media di permutazioni per frase considerata 4.43028611774199
